# Documents Similarity

### Read Files

In [1]:
#path_name_documents = './Databases/prova/prova.jsonl'
#path_name_documents = './Databases/prova/prova2.jsonl'
#path_name_documents = './Databases/prova/prova4.jsonl'
#path_name_documents = './Databases/prova/prova10.jsonl'
#path_name_documents = './Databases/prova/prova50.jsonl'  
#path_name_documents = './Databases/prova/prova5000.jsonl'
path_name_documents = './Databases/prova/prova2000.jsonl'
#path_name_documents = './Databases/prova/prova30000.jsonl'

In [2]:
import json
import numpy as np
import string

def readFile(path_name):
    # Load the JSONL file into a list
    with open(path_name, 'r') as f:
        lines = f.readlines()

    # Convert each JSON object into a dictionary
    dicts = [json.loads(line) for line in lines]

    # Convert the dictionaries into arrays and stack them vertically
    arrays = np.vstack([np.array(list(d.values())) for d in dicts])

    # Convert the arrays into a list of lists
    text = arrays.tolist()
    
    return text

documents = readFile(path_name_documents)

In [3]:
import time

def time_it(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Execution time: {end_time - start_time:.5f} seconds")
        return result
    return wrapper

## Tokenized

In [4]:
import json
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer


stop_words = set(stopwords.words('english'))

def stemmingLemming(filtered_tokens):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    # Perform stemming or lemmatization on filtered tokens
    
    filtered_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    filtered_tokens = [stemmer.stem(token) for token in filtered_tokens]

    return filtered_tokens
    
 
    

def tokenize(path_name):
    
    with open(path_name, "r") as f:
        data = f.readlines()

        # Create an empty list to store the tokenized documents
        tokenized_docs = []

        # Loop through each line in the JSONL file
        for line in data:
            # Parse the JSON string into a Python dictionary
            doc = json.loads(line)

            # Extract the text from the dictionary
            text = doc['text']
            text = text.lower()  # Convert to lowercase
            #text = re.sub(r'\d+', '', text)  # Remove all numbers
            text = text.translate(str.maketrans('', '', string.punctuation))  # Remove all punctuation

            # Tokenize the text using NLTK
            tokens = word_tokenize(text)
            tokensStemLem = stemmingLemming(tokens)

            # Add the tokenized document to the list
            tokenized_docs.append(tokensStemLem)

        # Print the tokenized documents
    return tokenized_docs


tokenized_docs = tokenize(path_name_documents)


## Sparse Vectors

### TF-IDF

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer


def calculateTFIDF(tokenized_docs):  
    vectorizer = TfidfVectorizer()
    # Fit and transform the tokenized documents into a TF-IDF matrix
    tfidf_matrix = vectorizer.fit_transform([' '.join(doc) for doc in tokenized_docs])

    # Get the feature names (tokens)
    feature_names = vectorizer.get_feature_names_out()

    # Return the TF-IDF matrix and the feature names
    return tfidf_matrix, feature_names,vectorizer
       
tfidf_matrix_docs, feature_names_docs,vectorizer  = calculateTFIDF(tokenized_docs)

## Cosine Similarity

In [6]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def similarity(tfidf_matrix):
    # we compute the cosine similarity between the documents
    cos_sim = cosine_similarity(tfidf_matrix)

    # we create a table with the similarity cosines for each pair of documents
    sim_table = pd.DataFrame(cos_sim, columns=['Doc ' + str(i+1) for i in range(cos_sim.shape[0])], index=['Doc ' + str(i+1) for i in range(cos_sim.shape[0])])
    
    return sim_table, cos_sim

cos_sim_table, cos_sim = similarity(tfidf_matrix_docs)
cos_sim_table

,Doc 1,Doc 2,Doc 3,Doc 4,Doc 5,Doc 6,Doc 7,Doc 8,Doc 9,Doc 10,...,Doc 1991,Doc 1992,Doc 1993,Doc 1994,Doc 1995,Doc 1996,Doc 1997,Doc 1998,Doc 1999,Doc 2000
Doc 1,1.000000,0.088140,0.076180,0.041710,0.135476,0.037048,0.079712,0.083859,0.046190,0.066090,...,0.052495,0.091297,0.074900,0.077853,0.055493,0.056206,0.155063,0.047254,0.098762,0.059326
Doc 2,0.088140,1.000000,0.216128,0.149980,0.125037,0.082548,0.081243,0.109634,0.165225,0.113318,...,0.136774,0.053780,0.086105,0.117846,0.103599,0.098679,0.084447,0.076612,0.107566,0.091730
Doc 3,0.076180,0.216128,1.000000,0.112982,0.164268,0.056795,0.076408,0.092951,0.092219,0.082467,...,0.088033,0.051083,0.078266,0.097425,0.101088,0.104249,0.106681,0.069464,0.115833,0.086805
Doc 4,0.041710,0.149980,0.112982,1.000000,0.063064,0.033588,0.038800,0.051888,0.088788,0.060705,...,0.094869,0.032597,0.046291,0.042923,0.047268,0.044550,0.043281,0.057974,0.074586,0.058088
Doc 5,0.135476,0.125037,0.164268,0.063064,1.000000,0.057647,0.097862,0.090730,0.087590,0.084105,...,0.101804,0.060967,0.101729,0.085532,0.103503,0.093017,0.073879,0.071080,0.152637,0.081881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Doc 1996,0.056206,0.098679,0.104249,0.044550,0.093017,0.050826,0.054918,0.069194,0.051968,0.064365,...,0.058417,0.054911,0.065844,0.067180,0.158831,1.000000,0.065041,0.064478,0.089159,0.082480
Doc 1997,0.155063,0.084447,0.106681,0.043281,0.073879,0.040495,0.064603,0.072925,0.048578,0.068262,...,0.052454,0.066905,0.070492,0.057624,0.063972,0.065041,1.000000,0.050453,0.091349,0.063624
Doc 1998,0.047254,0.076612,0.069464,0.057974,0.071080,0.048398,0.043551,0.061967,0.047952,0.049643,...,0.055534,0.038722,0.048710,0.054898,0.062131,0.064478,0.050453,1.000000,0.099591,0.050479
Doc 1999,0.098762,0.107566,0.115833,0.074586,0.152637,0.153502,0.071356,0.092917,0.074554,0.097069,...,0.085829,0.070660,0.104924,0.115240,0.085430,0.089159,0.091349,0.099591,1.000000,0.091079


### Set the threshold value

In [7]:
threshold = 0.3

# Sequential Algorithms

### For each document create pairs of similar documents

In [8]:
def sequential_pair_similar_documents(cos_sim, threshold):   
    # We create a list of pairs of similar documents with a similarity value greater than the threshold
    sim_pairs = []
    n_docs = cos_sim.shape[0]
    for i in range(n_docs):
        for j in range(i+1, n_docs):
            if cos_sim[i,j] > threshold:
                sim_pairs.append((i+1, j+1))
            
    return sim_pairs



### Best execution time

In [9]:
import time

def sequential_pair_similar_documents(cos_sim, threshold):
    start_time = time.time()
    # Find the indexes of cells that exceed the similarity threshold
    sim_idxs = np.argwhere(cos_sim > threshold)
    # Converts cell indexes to similar document pairs
    sim_pairs = [(i+1, j+1) for i, j in sim_idxs if i < j]
    
    end_time = time.time()
    sequential_total_time = end_time - start_time

    return sim_pairs, sequential_total_time


sim_pairs , sequential_total_time = sequential_pair_similar_documents(cos_sim, threshold)
sim_pairs
print(sequential_total_time)
    

0.016999006271362305


### For each document calculate the number of similar documents

In [10]:
def sequential_count_similar_documents(cos_sim, threshold):
    num_similar = []
    n_docs = cos_sim.shape[0]
    for i in range(n_docs):
        num = 0
        for j in range(i+1, n_docs):
            if cos_sim[i,j] > threshold:
                num += 1
        num_similar.append(num)
    return num_similar, threshold

num_similar, threshold = sequential_count_similar_documents(cos_sim, threshold)

### For each document create a list with similar documents

In [11]:
def sequential_similar_documents(cos_sim, threshold):
    similar_docs = []
    n_docs = cos_sim.shape[0]
    for i in range(n_docs):
        sim_docs = []
        for j in range(i+1, n_docs):
            if cos_sim[i,j] > threshold:
                sim_docs.append(f"Doc {j+1}")
        similar_docs.append(sim_docs)
    return similar_docs

similar_docs = sequential_similar_documents(cos_sim, threshold)


### Table of final results

In [12]:
def create_similar_table(similar_docs, num_similar, threshold):
    doc_names = [f"Doc {i+1}" for i in range(len(num_similar))]
    similar_docs_str = [", ".join(docs) for docs in similar_docs]
    similar_table = pd.DataFrame({"Documents": doc_names, "Number of similar documents": num_similar, "Similar Documents": similar_docs_str, "Threshold": threshold})
    pd.set_option('display.max_rows', None)

    return similar_table

similar_table = create_similar_table(similar_docs, num_similar, threshold)
similar_table

,Documents,Number of similar documents,Similar Documents,Threshold
0,Doc 1,6,"Doc 639, Doc 1071, Doc 1077, Doc 1146, Doc 174...",0.3
1,Doc 2,0,,0.3
2,Doc 3,1,Doc 1629,0.3
3,Doc 4,0,,0.3
4,Doc 5,1,Doc 98,0.3
5,Doc 6,0,,0.3
6,Doc 7,2,"Doc 668, Doc 1194",0.3
7,Doc 8,0,,0.3
8,Doc 9,0,,0.3
9,Doc 10,0,,0.3


# Parallel Algorithm

### Function to take input data

In [13]:
from scipy.sparse import coo_matrix

def extract_document_terms(tfidf_matrix):
    matrix_coo = coo_matrix(tfidf_matrix)
    data = matrix_coo.data
    row = matrix_coo.row
    col = matrix_coo.col

    doc_terms = []
    current_doc = -1
    terms = []

    for i in range(len(data)):
        doc_id = row[i]
        term_id = col[i]
        term_value = data[i]

        if doc_id != current_doc:
            if terms:
                doc_terms.append((current_doc+1, terms))
                terms = []
            current_doc = doc_id

        terms.append((term_id, term_value))

    if terms:
        doc_terms.append((current_doc, terms))

    return doc_terms

doc_info_list = extract_document_terms(tfidf_matrix_docs)
#print(doc_info_list)


## Functions Map and Reduce

In [14]:
def my_map(doc_info):
    mapped_doc = []

    doc_id, terms = doc_info
    
    max_term_id = max(terms, key=lambda x: x[0])[0]
    doc_terms = [(term_id, value) for term_id, value in terms]
    mapped_doc.append((doc_id, max_term_id, doc_terms))

    return mapped_doc



In [15]:
def apply_my_map(doc_info_list):
    total_mapped = []

    for doc_info in doc_info_list:
        mapped = my_map(doc_info)
        total_mapped.extend(mapped)

    return total_mapped


In [16]:
def my_reduce(docs, threshold):
    pairs = []
    docs_list = list(docs)  # Convert the docs iterator to a list

    n_docs = len(docs_list)
    print(n_docs)
    
    for i in range(n_docs - 1):
        for j in range(i + 1, n_docs):
            doc1_id, term_id, doc1 = docs_list[i]
            doc2_id, _, doc2 = docs_list[j]

            terms_1 = {t_id1: val1 for t_id1, val1 in doc1}
            terms_2 = {t_id2: val2 for t_id2, val2 in doc2}

            common_terms = set(terms_1).intersection(terms_2)

            if not common_terms:
                continue

            max_term = max(common_terms)

            if term_id != max_term:
                continue

            sim = 0.0

            for term in common_terms:
                sim += terms_1[term] * terms_2[term]

            if sim >= threshold:
                if doc2_id == n_docs:
                    doc2_id += 1
                pair = ((doc1_id, doc2_id), sim)
                pairs.append(pair)

    return pairs




# Implementazioni per Spark 

In [17]:
import os
os.environ['PYSPARK_PYTHON'] = 'C:/Users/lita4/anaconda3/python.exe'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'C:/Users/lita4/anaconda3/python.exe'

### Execution with spark

#### Input Data

In [18]:
doc_info_list = extract_document_terms(tfidf_matrix_docs)

In [33]:
from pyspark import SparkConf, SparkContext
import time

total_results_time = []



for i in range(1,9):
    print("EXECUTION WITH PARTITIONS = ")
    start_time = time.time()
    conf = SparkConf().setAppName("MyApp").setMaster("local[16]").set("spark.executor.memory", "20g").set("spark.network.timeout", "600s")
    sc = SparkContext(conf=conf)
    

    print("EXECUTION INPUT")
    input_rdd = sc.parallelize(doc_info_list,i)
    #print(input_rdd.collect())

    print("EXECUTION MAP")
    mapped_rdd = input_rdd.flatMap(my_map)
    #print(mapped_rdd.collect())
    
    print("EXECUTION REPARTITION")
    mapped_rdd = mapped_rdd.repartition(i)
    # Repartition the RDD to evenly distribute the data across partitions


    print("EXECUTION REDUCE")
    threshold = 0.5 # Replace with the desired threshold value
    reduced_pairs = mapped_rdd.mapPartitions(lambda docs: my_reduce(docs, threshold))
    print("EXECUTION RESULTS")
    results = reduced_pairs.collect()

    end_time = time.time()
    total_time = end_time - start_time

    print("Time of execution",total_time)
    total_results_time.append(total_time)
    
    results = list(set(results))
    for result in sorted(results):
        print(result)
    
    print()
    
    sc.stop()

EXECUTION INPUT
EXECUTION MAP
EXECUTION REPARTITION
EXECUTION REDUCE
EXECUTION RESULTS
Time of execution 66.22253847122192
((36, 140), 0.50578427621122)
((92, 574), 0.5102491003148797)
((104, 1524), 0.5216489704838196)
((124, 1621), 0.5783395553835083)
((145, 628), 0.5397341647704766)
((207, 347), 0.5129237170266431)
((307, 472), 0.550784702575116)
((321, 501), 0.602878473261898)
((336, 651), 0.5656901503888864)
((336, 986), 0.5488596472095627)
((347, 586), 0.5248328881898989)
((347, 647), 0.559156156806872)
((449, 774), 0.520372316149544)
((455, 706), 0.5495113607868259)
((467, 665), 0.536761926821571)
((475, 1825), 0.5958797040084671)
((480, 1699), 0.8189592369975276)
((574, 1250), 0.555401348019002)
((580, 1009), 0.5105937205338915)
((653, 1439), 0.6021463327436744)
((726, 1331), 0.5930624798172495)
((829, 1044), 0.5604888856700294)
((829, 1930), 0.5482495629731977)
((866, 1116), 0.6748417559590317)
((904, 1140), 0.5143012797477065)
((1044, 1079), 0.622843561919955)
((1044, 1768), 0

In [22]:
sc.stop()

In [32]:
total_results_time

[66.57488441467285,
 21.458228588104248,
 13.837072849273682,
 11.665378332138062,
 11.133236169815063,
 11.770022869110107,
 12.515515565872192,
 13.402557373046875]

In [28]:
time_partition2000Core2 = total_results_time

[62.33458971977234,
 18.17539954185486,
 16.412301778793335,
 11.637999534606934,
 11.045621871948242,
 9.60500192642212,
 9.568810939788818,
 9.155786752700806]

In [26]:
time_partition2000  #8 cores

[62.41481018066406,
 17.889978647232056,
 10.518076181411743,
 8.338082075119019,
 7.705981254577637,
 7.5420002937316895,
 7.690092325210571,
 7.9210004806518555]